In [2]:
from train import load_data

train_loader, test_loader = load_data()

Loading test_df from file
Loading train_df from file
Test data loading complete
Train data loading complete


In [6]:
import torch
from torchvision import models, transforms
from torch.utils.data import DataLoader
from PIL import Image
import numpy as np

resnet = models.resnet50(pretrained=True)
resnet = torch.nn.Sequential(*list(resnet.children())[:-1]) 
resnet.eval() 
resnet = resnet.cuda() 

preprocess = transforms.Compose([
                transforms.Resize((224, 224)), 
                transforms.ToTensor(),
                transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
            ])

/home/deependra/.conda/envs/nlrc/lib/python3.7/site-packages/torchvision/models/_utils.py:209: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  f"The parameter '{pretrained_param}' is deprecated since 0.13 and may be removed in the future, "
/home/deependra/.conda/envs/nlrc/lib/python3.7/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [7]:
def extract_features(loader, model, batch_size=32, collate=None):

    features = []
    with torch.no_grad():
        for imgs, targets in loader:
            imgs = torch.stack(imgs[0]).cuda()
            feats = model(imgs)
            features.append(feats.cpu().numpy())
    return np.vstack(features)

train_features = extract_features(train_loader, resnet)

In [8]:
test_features = extract_features(test_loader, resnet)

AttributeError: Caught AttributeError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "/home/deependra/.conda/envs/nlrc/lib/python3.7/site-packages/torch/utils/data/_utils/worker.py", line 302, in _worker_loop
    data = fetcher.fetch(index)
  File "/home/deependra/.conda/envs/nlrc/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 58, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/deependra/.conda/envs/nlrc/lib/python3.7/site-packages/torch/utils/data/_utils/fetch.py", line 58, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
  File "/home/deependra/kuyesera/dahircnn/dataset.py", line 423, in __getitem__
    img = F.resize(img, self.resize_size)
AttributeError: module 'torch.nn.functional' has no attribute 'resize'


In [ ]:
train_features_new = train_features.squeeze(-1).squeeze(-1)

In [ ]:
test_features_new = test_features.squeeze(-1).squeeze(-1)

In [ ]:
from sklearn.manifold import TSNE
import matplotlib.pyplot as plt
import numpy as np


features = np.concatenate([train_features_new, test_features_new])
labels = np.concatenate([np.zeros(len(train_features_new)), np.ones(len(test_features_new))]) 

tsne = TSNE(n_components=2, random_state=42, perplexity=30)
reduced_features_tsne = tsne.fit_transform(features)

plt.figure(figsize=(10, 8))
plt.scatter(
    reduced_features_tsne[labels == 0, 0], 
    reduced_features_tsne[labels == 0, 1], 
    c='blue', label='Train Set', alpha=0.5
)
plt.scatter(
    reduced_features_tsne[labels == 1, 0], 
    reduced_features_tsne[labels == 1, 1], 
    c='red', label='Test Set', alpha=0.5
)
plt.title('t-SNE Visualization of Features')
plt.xlabel('t-SNE Component 1')
plt.ylabel('t-SNE Component 2')
plt.legend()
plt.show()
